#### 1.	Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8.   

a.	It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.
b.	Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.
c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  
d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.  


In [1]:
import requests
import nltk 
import re
from bs4 import BeautifulSoup
from urllib import request
from requests import get

In [2]:
url = 'https://www.imdb.com/list/ls095374765/?st_dt=&mode=grid&page=1&ref_=ttls_ref_gnr&genres=Comedy&sort=moviemeter,asc'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
def Reviewslinks(website):
    soup = BeautifulSoup(website, 'html.parser')
    
    
    f = soup.find_all('div', class_='ipl-icon ipl-star-icon')
    f = soup.find_all('span', class_="rating-other-user-rating")#.find('span').text

    idxPos = -1
    idxNeg = -1

    for idx, i in enumerate(f):
        sp = (f[idx].text.split('/'))
        sp = int(sp[0][-2:])
        if sp > 8:
            idxPos = idx
        elif sp < 4:
            idxNeg = idx
    return([idxNeg,idxPos])

### 2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:
a.	In Python, use BeautifulSoup to grab the main review text from each link.  
b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 
c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.


In [4]:
NPchunker = []
POSTaging = []

In [5]:
for div in html_soup.findAll('div', attrs={'class':'title'}):
    if len(NPchunker) > 98:
        break
    else:
        url = 'https://www.imdb.com' + div.find('a')['href'] 
        url = url + 'reviews'
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        review_containers = html_soup.find_all('div', class_='text show-more__control')

        reviewIndex = Reviewslinks(response.content)
        reviews = []

        if reviewIndex[0] == -1 or reviewIndex[1] == -1:
            pass
        else:
            reviews.append(review_containers[reviewIndex[0]].text)
            reviews.append(review_containers[reviewIndex[1]].text)
            count =+ 2
# This for statement allows for there reviews to be POS tagged and chunked 
            for i in reviews:
                text = nltk.word_tokenize(i)
                Taggedsentence = nltk.pos_tag(text)
                POSTaging.append(Taggedsentence)
                # Regex 
                grammar = "NP: {<DT>?<JJ>*<NN>}"

                cp = nltk.RegexpParser(grammar)
                result = cp.parse(Taggedsentence)
                NPchunker.append(result)

In [6]:
# Here we can see the chunks that were formed from NLTK going through the lis 
print(NPchunker) 

[Tree('S', [('I', 'PRP'), ('have', 'VBP'), ("n't", 'RB'), ('seen', 'VBN'), Tree('NP', [('this', 'DT'), ('movie', 'NN')]), ('for', 'IN'), Tree('NP', [('a', 'DT'), ('long', 'JJ'), ('time', 'NN')]), (',', ','), ('so', 'RB'), ('for', 'IN'), ('all', 'DT'), ('I', 'PRP'), ('know', 'VBP'), ('it', 'PRP'), ("'s", 'VBZ'), ('horrible', 'JJ'), (',', ','), ('but', 'CC'), ('I', 'PRP'), ('have', 'VBP'), Tree('NP', [('nothing', 'NN')]), ('but', 'CC'), ('good', 'JJ'), ('memories', 'NNS'), ('of', 'IN'), Tree('NP', [('it.Any', 'JJ'), ('red-blooded', 'JJ'), ('American', 'JJ'), ('male', 'NN')]), ('has', 'VBZ'), ('to', 'TO'), ('at', 'IN'), ('least', 'JJS'), ('appreciate', 'VB'), Tree('NP', [('the', 'DT'), ('premise', 'NN')]), (':', ':'), ('two', 'CD'), Tree('NP', [('horny', 'NN')]), ('teens', 'NNS'), ('discover', 'VBP'), Tree('NP', [('a', 'DT'), ('way', 'NN')]), ('to', 'TO'), ('create', 'VB'), Tree('NP', [('the', 'DT'), ('perfect', 'JJ'), ('woman', 'NN')]), (',', ','), ('who', 'WP'), ('exists', 'VBZ'), ('for

In [7]:
 # Here we can verify the right number of reviews were extracted with the crawler
print(len(NPchunker))

100


#### 3.	Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).


 The genre that I selected was comedy. The place that I went for reviews was IDMb. I collected 100 and used NLTK to do the chunking and POS tagging. Beautiful soup was used to establish connection. Then using page inspection on the I foud the right link to put into BSoup to  pull out the links for the reviews. Then iterated thorugh the link while appending the list. Finally Using the NLTK to do the chunking and POS tagging.